In [11]:
import os 
import sys
import datetime
import csv 

import cirq
import qsimcirq
import openfermion
import numpy as np
import multiprocessing as mp
from functools import partial

Pi=3.1415

## anzats

In [12]:
class AnzatsAFMHeisenberg_2d:
    def __init__(self, length, gamma, beta):
        rows = 2
        cols = int(length / 2)
        if rows * cols % 2 != 0:
            raise ValueError("The number of qubits (rows * cols) must be even.")

        # Initialize circuit
        circuit = cirq.Circuit()
        qubits = [cirq.GridQubit(i, j) for i in range(rows) for j in range(cols)]

        # Apply initial gates and CNOT gates for correlations
        for i in range(rows):
            for j in range(cols):
                index = i * cols + j
                if index % 2 == 0:  # Even qubits
                    circuit.append(cirq.H(qubits[index]))
                    circuit.append(cirq.Y(qubits[index]))
                else:  # Odd qubits
                    circuit.append(cirq.X(qubits[index]))

                # Apply CNOT gates for correlations with neighbors
                if j + 1 < cols:  # Right neighbor
                    circuit.append(cirq.CNOT(qubits[index], qubits[index + 1]))
                if i + 1 < rows:  # Down neighbor
                    circuit.append(cirq.CNOT(qubits[index], qubits[index + cols]))
                if j - 1 >= 0:  # Left neighbor
                    circuit.append(cirq.CNOT(qubits[index], qubits[index - 1]))
                if i - 1 >= 0:  # Up neighbor
                    circuit.append(cirq.CNOT(qubits[index], qubits[index - cols]))

        # Add gamma circuit
        for index in range(len(gamma)):
            for i in range(rows):
                for j in range(cols - 1):
                    # Apply XX, YY, ZZ gates to horizontal neighbors
                    circuit.append(
                        cirq.XX(qubits[i * cols + j], qubits[i * cols + j + 1]) ** (-gamma[index] * 2 / Pi)
                    )
                    circuit.append(
                        cirq.YY(qubits[i * cols + j], qubits[i * cols + j + 1]) ** (-gamma[index] * 2 / Pi)
                    )
                    circuit.append(
                        cirq.ZZ(qubits[i * cols + j], qubits[i * cols + j + 1]) ** (-gamma[index] * 2 / Pi)
                    )

            for j in range(cols):
                for i in range(rows - 1):
                    # Apply XX, YY, ZZ gates to vertical neighbors
                    circuit.append(
                        cirq.XX(qubits[i * cols + j], qubits[(i + 1) * cols + j]) ** (-gamma[index] * 2 / Pi)
                    )
                    circuit.append(
                        cirq.YY(qubits[i * cols + j], qubits[(i + 1) * cols + j]) ** (-gamma[index] * 2 / Pi)
                    )
                    circuit.append(
                        cirq.ZZ(qubits[i * cols + j], qubits[(i + 1) * cols + j]) ** (-gamma[index] * 2 / Pi)
                    )

            # Add beta circuit
            for i in range(rows):
                for j in range(cols - 1):
                    # Apply XX, YY, ZZ gates to horizontal neighbors
                    circuit.append(
                        cirq.XX(qubits[i * cols + j], qubits[i * cols + j + 1]) ** (-beta[index] * 2 / Pi)
                    )
                    circuit.append(
                        cirq.YY(qubits[i * cols + j], qubits[i * cols + j + 1]) ** (-beta[index] * 2 / Pi)
                    )
                    circuit.append(
                        cirq.ZZ(qubits[i * cols + j], qubits[i * cols + j + 1]) ** (-beta[index] * 2 / Pi)
                    )

            for j in range(cols):
                for i in range(rows - 1):
                    # Apply XX, YY, ZZ gates to vertical neighbors
                    circuit.append(
                        cirq.XX(qubits[i * cols + j], qubits[(i + 1) * cols + j]) ** (-beta[index] * 2 / Pi)
                    )
                    circuit.append(
                        cirq.YY(qubits[i * cols + j], qubits[(i + 1) * cols + j]) ** (-beta[index] * 2 / Pi)
                    )
                    circuit.append(
                        cirq.ZZ(qubits[i * cols + j], qubits[(i + 1) * cols + j]) ** (-beta[index] * 2 / Pi)
                    )

        self.circuit = circuit
        self.qubits = qubits
        self.gamma = gamma
        self.beta = beta
        
        # Function to visualize the circuit using matplotlib
        # def visualize_circuit(circuit):
        #     diagram = cirq.CircuitDiagramInfo(circuit)
        #     fig, ax = plt.subplots(figsize=(12, 6))
        #     cirq.circuits.circuit_diagram.draw_circuit_diagram(ax, diagram)
        #     plt.show()

        # # Visualize the circuit
        # visualize_circuit(ansatz.circuit) 
           
    def circuit_to_latex_using_qcircuit(self):
        return cirq.contrib.circuit_to_latex_using_qcircuit(
            self.circuit, self.qubits
        )

## expectation

In [13]:
class AFMHeisenbergArgs():
    def __init__(self, length, qsim_option):
        self.length = length
        self.qsim_option = qsim_option

def get_expectation_afm_heisenberg_new_symmetry(function_args, gamma, beta):
    # This function calculates the expectation value for the AFM Heisenberg model with a new symmetry
    # using a 2D quantum circuit ansatz.

    # Initialize the ansatz for the 2D AFM Heisenberg model with given parameters
    anzats = AnzatsAFMHeisenberg_2d(function_args.length, gamma, beta)
    circuit = anzats.circuit
    qubits = anzats.qubits

    # Initialize the quantum simulator
    simulator = qsimcirq.QSimSimulator(function_args.qsim_option)

    # Simulate the circuit to get the initial state vector
    vector = simulator.simulate(circuit).state_vector()

    value = 0 + 0j  # Initialize the expectation value as a complex number

    rows = 2  # Number of rows in the 2D grid of qubits
    cols = int(function_args.length / 2)  # Number of columns in the 2D grid of qubits

    # Iterate over the 2D grid of qubits
    for i in range(rows):
        for j in range(cols - 1):
            # Horizontal neighbors

            # Create copies of the original circuit for each Pauli operator (X, Y, Z)
            circuitX = circuit.copy()
            circuitY = circuit.copy()
            circuitZ = circuit.copy()

            # Apply Pauli-X operators to the horizontal neighbors and simulate the circuit
            circuitX.append(cirq.X(qubits[i * cols + j]))
            circuitX.append(cirq.X(qubits[i * cols + j + 1]))
            vector2 = simulator.simulate(circuitX).state_vector()
            value += np.dot(vector2.conj(), vector)  # Add the overlap to the expectation value
            print(f'Horizontal: Applying Pauli-X operators to {qubits[i * cols + j]} and {qubits[i * cols + j + 1]}')
            
            # Apply Pauli-Y operators to the horizontal neighbors and simulate the circuit
            circuitY.append(cirq.Y(qubits[i * cols + j]))
            circuitY.append(cirq.Y(qubits[i * cols + j + 1]))
            vector2 = simulator.simulate(circuitY).state_vector()
            value += np.dot(vector2.conj(), vector)  # Add the overlap to the expectation value
            print(f'Horizontal: Applying Pauli-Y operators to {qubits[i * cols + j]} and {qubits[i * cols + j + 1]}')
            
            # Apply Pauli-Z operators to the horizontal neighbors and simulate the circuit
            circuitZ.append(cirq.Z(qubits[i * cols + j]))
            circuitZ.append(cirq.Z(qubits[i * cols + j + 1]))
            vector2 = simulator.simulate(circuitZ).state_vector()
            value += np.dot(vector2.conj(), vector)  # Add the overlap to the expectation value
            print(f'Horizontal: Applying Pauli-Z operators to {qubits[i * cols + j]} and {qubits[i * cols + j + 1]}')
            
    for i in range(rows - 1):
        for j in range(cols):
            # Vertical neighbors

            # Create copies of the original circuit for each Pauli operator (X, Y, Z)
            circuitX = circuit.copy()
            circuitY = circuit.copy()
            circuitZ = circuit.copy()

            # Apply Pauli-X operators to the vertical neighbors and simulate the circuit
            circuitX.append(cirq.X(qubits[i * cols + j]))
            circuitX.append(cirq.X(qubits[(i + 1) * cols + j]))
            vector2 = simulator.simulate(circuitX).state_vector()
            value += np.dot(vector2.conj(), vector)  # Add the overlap to the expectation value
            print(f'Vertical: Applying Pauli-X operators to {qubits[i * cols + j]} and {qubits[i * cols + j + 1]}')
            
            # Apply Pauli-Y operators to the vertical neighbors and simulate the circuit
            circuitY.append(cirq.Y(qubits[i * cols + j]))
            circuitY.append(cirq.Y(qubits[(i + 1) * cols + j]))
            vector2 = simulator.simulate(circuitY).state_vector()
            value += np.dot(vector2.conj(), vector)  # Add the overlap to the expectation value
            print(f'Vertical: Applying Pauli-Y operators to {qubits[i * cols + j]} and {qubits[i * cols + j + 1]}')
            
            # Apply Pauli-Z operators to the vertical neighbors and simulate the circuit
            circuitZ.append(cirq.Z(qubits[i * cols + j]))
            circuitZ.append(cirq.Z(qubits[(i + 1) * cols + j]))
            vector2 = simulator.simulate(circuitZ).state_vector()
            value += np.dot(vector2.conj(), vector)  # Add the overlap to the expectation value
            print(f'Vertical: Applying Pauli-Z operators to {qubits[i * cols + j]} and {qubits[i * cols + j + 1]}')
            
    # Return the real part of the expectation value
    return np.real(value)

## optimization

In [14]:
def get_gradient(function, gamma: np.array, beta: np.array, delta_gamma, delta_beta, iter):
    grad_gamma = np.zeros_like(gamma)
    grad_beta  = np.zeros_like(beta)
    gamma_edge = gamma
    beta_edge  = beta
    # initial gamma, beta?
    
    if not (gamma.size == beta.size):
        return 1

    for index in range(gamma.size):
        center = gamma[index]
        gamma_edge[index] = gamma[index] - delta_gamma
        e1 = function(gamma=gamma_edge, beta=beta)
        gamma_edge[index] = gamma[index] + delta_gamma
        e2 = function(gamma=gamma_edge, beta=beta)
        grad_gamma[index] = (e2.real-e1.real)/(2*delta_gamma)
        gamma[index] = center

        center = beta[index]
        beta_edge[index] = beta[index] - delta_beta
        e1 = function(gamma=gamma, beta=beta_edge)
        beta_edge[index] = beta[index] + delta_beta
        e2 = function(gamma=gamma, beta=beta_edge)
        grad_beta[index] = (e2.real-e1.real)/(2*delta_beta)
        beta[index] = center
    
    return grad_gamma, grad_beta


def optimize_by_gradient_descent(function, initial_gamma: np.array, initial_beta: np.array, alpha, delta_gamma, delta_beta, iteration, figure=True,filepath=""):
    gamma, beta = initial_gamma, initial_beta

    textlines = []
    headline = ["iter", "energy"]
    for p in range(int(len(initial_gamma))):
        headline.append("gamma[{}]".format(p))
        headline.append("bata[{}]".format(p))
    print(headline)
    textlines.append(headline)

    for iter in range(int(iteration)):
        # it is complex for me to set get_gradient for two optical parameter_vector
        grad_gamma, grad_beta = get_gradient(function, gamma, beta, delta_gamma, delta_beta, iter)
        gamma -= alpha * grad_gamma
        beta  -= alpha * grad_beta
        energy = function(gamma=gamma, beta=beta)

        record = [iter, energy]
        for index in range(gamma.size):
            record.append(gamma[index])
            record.append(beta[index])
        textlines.append(record)
        print(record)
    
    if len(filepath)>0:
        with open(filepath, mode='a') as f:
            writer = csv.writer(f)
            for i, textline in enumerate(textlines):
                writer.writerow(textline)
                # f.write("{}\n".format(textline))

    return gamma, beta

## case: length 8, optimize gamma and beta

In [18]:
length = 8
gamma = np.array([0.6, 0.6, 0.6, 0.6])
beta  = np.array([0.6, 0.6, 0.6, 0.6])
qsim_option = {'t': int(length/2), 'f':1}
function_args = AFMHeisenbergArgs(length, qsim_option)

anzats = AnzatsAFMHeisenberg_2d(length, gamma, beta)
circuit = anzats.circuit
print(circuit)

simulator = cirq.Simulator()
result = simulator.simulate(circuit, qubit_order=anzats.qubits)
vector = cirq.final_state_vector(circuit)
print(result)

# norm = np.dot(vector.conjugate(), vector)
# print("norm: {}".format(norm))

value = get_expectation_afm_heisenberg_new_symmetry(function_args,gamma, beta)
# print("expectation on afm: {}".format(value))

initial_gamma = np.array([0.6, 0.6, 0.6, 0.6])
initial_beta  = np.array([0.6, 0.6, 0.6, 0.6])
iteration = 10
alpha = 0.1
delta_gamma = 0.0001
delta_beta  = 0.0001
gamma, beta = optimize_by_gradient_descent(partial(get_expectation_afm_heisenberg_new_symmetry, function_args=function_args), initial_gamma, initial_beta, alpha, delta_gamma, delta_beta, iteration)

print(gamma, beta)

                       ┌──┐       ┌──┐       ┌──┐   ┌──┐   ┌──┐                                   ┌──────────────────┐               ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐                           ┌───────────────────────────┐   ┌───────────────────────────┐   ┌───────────────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌───────────────────────────┐   ┌───────────────────────────┐   ┌───────────────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌───────────────────────────┐   ┌───────────────────────────┐   ┌───────────────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌──────────────────┐   ┌─────────────────────────